# High-level Comparison Between the Network Covariance and Characteristic Matrices

In this notebook, we provide a rough comparison between covariance-based and entropy-based topology inference methods.
The goal is to test each method across a range of entangled states to get an idea of their relative performance.


## Key Differences

### Covariance Matrix Advantages

There is a significant performance advantage over the characteristic matrix:
* This can easily be observed by noting the cell execution times, especially when more qubits are considered. 
* The covariance is evaluated from one circuit evaluation $O(1)$, whereas the charcteristic matrix requires a minimum of $O(n^2)$ circuit evaluations where $n$ is the number of qubits. Given clever parallelization in quantum circuits, the characteristic matrix may be evaluated in $O(n)$.
* Optimizing the covariance matrix seems to require significantly fewer steps than needed to successfully infer topology using the characteristic matrix. This means that fewer circuits need to be run.
* The optimization appears to be more robust and less susceptible to local minima. We see this in the fact that the characteristic matrix often fails to infer the GHZ states with more than 3 qubits.


### Characteristic Matrix Advantages

A key difference between the covariance and characteristic matrices is that the covariance matrix seems to always have ones along the diagonal, even if the measured qubit is not correlated with any other qubits. On the contrary, the characteristic matrix measures a zero von neumann entropy if there is no correlation whatsoever.

The threshold of deciding if a correlation exists needs to be raised as the number of shots decreases.
The rate at which covariance methods need to be raised seems to more than characteristic matrices.
This may mean, that the variance of the entropic quantities in the low shot regime are improved over those of the covariance matrix.

In [4]:
import qnetti
import pennylane as qml
from pennylane import numpy as qnp
import qnetvo

print(qnetvo.__version__)
print(qml.__version__)

0.4.1
0.28.0


In [5]:
def decision_matrix(mat, atol=0.1):
    return qnp.where(qnp.abs(mat) > atol, 1, 0)


def matrix_distance(mat1, mat2):
    return qnp.linalg.norm(qnp.array(mat1) - qnp.abs(qnp.array(mat2)))


def characteristic_matrix_inference(prep_node, expected_mat, **kwargs):
    char_mat = qnetti.qubit_characteristic_matrix(prep_node, **kwargs)
    dec_mat = decision_matrix(char_mat)
    char_dist = matrix_distance(expected_mat, char_mat)

    print("characteristic matrix :\n", char_mat)
    print("decision matrix :\n", dec_mat)
    print("expected matrix :\n", expected_mat)
    print("distance to expected : ", char_dist)


def covariance_matrix_inference(
    prep_node,
    num_qubits,
    expected_mat,
    meas_wires=None,
    step_size=0.1,
    num_steps=10,
    verbose=False,
    shots=None,
    qnode_kwargs={},
    atol=0.1,
):
    settings = qnp.random.rand(3 * num_qubits, requires_grad=True)
    cov_cost = qnetti.qubit_covariance_cost_fn(
        prep_node, meas_wires=meas_wires, shots=shots, qnode_kwargs=qnode_kwargs
    )

    settings_list = [settings]
    cost_vals = [cov_cost(settings)]

    opt = qml.GradientDescentOptimizer(stepsize=step_size)
    for i in range(num_steps):
        settings, cost_val = opt.step_and_cost(cov_cost, settings)

        cost_vals += [cost_val]
        settings_list += [settings]

    min_id = qnp.argmin(cost_vals)
    min_cost = cost_vals[min_id]
    opt_settings = settings_list[min_id]

    cov_mat = qnetti.qubit_covariance_matrix_fn(
        prep_node, meas_wires=meas_wires, shots=shots, qnode_kwargs=qnode_kwargs
    )(opt_settings)
    dec_mat = decision_matrix(cov_mat, atol=atol)
    cov_dist = matrix_distance(expected_mat, cov_mat)

    print("covariance matrix :\n", cov_mat)
    print("decision matrix :\n", dec_mat)
    print("expected matrix :\n", expected_mat)
    print("distance to expected : ", cov_dist)

## Two-Qubit GHZ State

In [6]:
ghz2_mat = qnp.ones((2, 2))


def rot_bell_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])


bell_state_prep_node = qnetvo.PrepareNode(wires=[0, 1], ansatz_fn=rot_bell_state)

In [7]:
characteristic_matrix_inference(bell_state_prep_node, ghz2_mat, num_steps=10, step_size=0.1)

characteristic matrix :
 [[1.         0.99999999]
 [0.99999999 1.        ]]
decision matrix :
 [[1 1]
 [1 1]]
expected matrix :
 [[1. 1.]
 [1. 1.]]
distance to expected :  7.7269233365293e-09


In [8]:
covariance_matrix_inference(
    bell_state_prep_node,
    2,
    ghz2_mat,
    step_size=0.2,
    num_steps=6,
    verbose=False,
)

covariance matrix :
 [[ 1.         -0.99443667]
 [-0.99443667  1.        ]]
decision matrix :
 [[1 1]
 [1 1]]
expected matrix :
 [[1. 1.]
 [1. 1.]]
distance to expected :  0.007867731578690097


## 3-Qubit GHZ state

In [6]:
ghz3_mat = qnp.ones((3, 3))


def rot_ghz3_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])


ghz3_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2], ansatz_fn=rot_ghz3_state)

In [7]:
characteristic_matrix_inference(ghz3_state_prep_node, ghz3_mat, num_steps=10, step_size=0.1)

characteristic matrix :
 [[1.         0.26217712 0.32438733]
 [0.26217712 1.         0.48511411]
 [0.32438733 0.48511411 1.        ]]
decision matrix :
 [[1 1 1]
 [1 1 1]
 [1 1 1]]
expected matrix :
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
distance to expected :  1.5911898432287723


In [8]:
covariance_matrix_inference(
    ghz3_state_prep_node,
    3,
    ghz3_mat,
    step_size=0.1,
    num_steps=10,
    verbose=False,
)

covariance matrix :
 [[ 1.         -0.93711714  0.9407836 ]
 [-0.93711714  1.         -0.99456996]
 [ 0.9407836  -0.99456996  1.        ]]
decision matrix :
 [[1 1 1]
 [1 1 1]
 [1 1 1]]
expected matrix :
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
distance to expected :  0.12239543166992822


## 4-Qubit GHZ

In [6]:
ghz4_mat = qnp.ones((4, 4))


def rot_ghz4_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])


ghz4_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2, 3], ansatz_fn=rot_ghz4_state)

In [7]:
characteristic_matrix_inference(ghz4_state_prep_node, ghz4_mat, num_steps=30, step_size=0.1)

iteration :  0 , score :  0.213121392982631
elapsed time :  0.0251009464263916
iteration :  1 , score :  0.27278998771866547
elapsed time :  0.024473905563354492
iteration :  2 , score :  0.35455003632244386
elapsed time :  0.022964000701904297
iteration :  3 , score :  0.46845034912247385
elapsed time :  0.020957231521606445
iteration :  4 , score :  0.6295243002881568
elapsed time :  0.023871183395385742
iteration :  5 , score :  0.8598673372771539
elapsed time :  0.026052236557006836
iteration :  6 , score :  1.1903761770205195
elapsed time :  0.02522897720336914
iteration :  7 , score :  1.6592382678234787
elapsed time :  0.023853063583374023
iteration :  8 , score :  2.299511941885486
elapsed time :  0.022566795349121094
iteration :  9 , score :  3.1040051059350136
elapsed time :  0.02387404441833496
iteration :  10 , score :  3.9759810124747355
elapsed time :  0.02502608299255371
iteration :  11 , score :  4.746871940778391
elapsed time :  0.027704954147338867
iteration :  12 , s

In [8]:
covariance_matrix_inference(
    ghz4_state_prep_node,
    4,
    ghz4_mat,
    step_size=0.2,
    num_steps=6,
    verbose=False,
)

covariance matrix :
 [[ 1.          0.55915228  0.99475324 -0.68402132]
 [ 0.55915228  1.          0.56196958 -0.38642667]
 [ 0.99475324  0.56196958  1.         -0.68746778]
 [-0.68402132 -0.38642667 -0.68746778  1.        ]]
decision matrix :
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
distance to expected :  1.3858109774375191


# 5-Qubit GHZ

In [9]:
ghz5_mat = qnp.ones((5, 5))


def rot_ghz5_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[4]])


ghz5_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2, 3, 4], ansatz_fn=rot_ghz5_state)

In [10]:
characteristic_matrix_inference(ghz5_state_prep_node, ghz5_mat, num_steps=30, step_size=0.1)

iteration :  0 , score :  1.2317907909238186
elapsed time :  0.03287005424499512
iteration :  1 , score :  1.5735993381145634
elapsed time :  0.029414892196655273
iteration :  2 , score :  2.054118231307376
elapsed time :  0.029582977294921875
iteration :  3 , score :  2.72405582340192
elapsed time :  0.02910780906677246
iteration :  4 , score :  3.5976217588841415
elapsed time :  0.041326045989990234
iteration :  5 , score :  4.585151994626493
elapsed time :  0.0818021297454834
iteration :  6 , score :  5.52326705338041
elapsed time :  0.030249834060668945
iteration :  7 , score :  6.37577235008391
elapsed time :  0.03187704086303711
iteration :  8 , score :  7.109212192796372
elapsed time :  0.033980369567871094
iteration :  9 , score :  7.558930917337812
elapsed time :  0.030277013778686523
iteration :  10 , score :  7.817772470127359
elapsed time :  0.0319218635559082
iteration :  11 , score :  8.0687332132152
elapsed time :  0.03290700912475586
iteration :  12 , score :  8.3985456

In [14]:
covariance_matrix_inference(
    ghz5_state_prep_node, 5, ghz5_mat, step_size=0.1, num_steps=10, verbose=False
)

covariance matrix :
 [[ 1.          0.53899521  0.9999711  -0.99131748 -0.99996745]
 [ 0.53899521  1.          0.53901079 -0.53434626 -0.53900882]
 [ 0.9999711   0.53901079  1.         -0.99134613 -0.99999635]
 [-0.99131748 -0.53434626 -0.99134613  1.          0.99134252]
 [-0.99996745 -0.53900882 -0.99999635  0.99134252  1.        ]]
decision matrix :
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
expected matrix :
 [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
distance to expected :  1.3073698562073244


# 8-qubit GHZ state

In [11]:
ghz8_mat = qnp.ones((8, 8))


def rot_ghz8_state(settings, wires):
    qnetvo.ghz_state(settings, wires)
    qml.ArbitraryUnitary([qnp.pi / 4, 0.3, -1.5], wires=[wires[0]])
    qml.ArbitraryUnitary([-0.7, 2.9, 1.2], wires=[wires[1]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[2]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[3]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[4]])
    qml.ArbitraryUnitary([-0.9, 0.9, 0.12], wires=[wires[5]])
    qml.ArbitraryUnitary([2, 2, 2], wires=[wires[6]])
    qml.ArbitraryUnitary([1, 2.3, 0.4], wires=[wires[7]])


ghz8_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=rot_ghz8_state,
)

In [12]:
characteristic_matrix_inference(ghz8_state_prep_node, ghz8_mat, num_steps=30, step_size=0.1)

iteration :  0 , score :  2.967336791428753
elapsed time :  0.0642390251159668
iteration :  1 , score :  4.388716738106083
elapsed time :  0.062355995178222656
iteration :  2 , score :  6.430286103564649
elapsed time :  0.08305716514587402
iteration :  3 , score :  9.473165995399842
elapsed time :  0.12350988388061523
iteration :  4 , score :  13.955833132861654
elapsed time :  0.0678260326385498
iteration :  5 , score :  18.481583544354578
elapsed time :  0.07754230499267578
iteration :  6 , score :  20.110974683829458
elapsed time :  0.06719398498535156
iteration :  7 , score :  21.12713012134492
elapsed time :  0.07544612884521484
iteration :  8 , score :  22.999521168810183
elapsed time :  0.0652010440826416
iteration :  9 , score :  25.904124167711498
elapsed time :  0.12159204483032227
iteration :  10 , score :  25.833449032880615
elapsed time :  0.06905889511108398
iteration :  11 , score :  25.15618975325883
elapsed time :  0.07168388366699219
iteration :  12 , score :  24.8642

In [17]:
covariance_matrix_inference(
    ghz8_state_prep_node,
    8,
    ghz8_mat,
    step_size=0.1,
    num_steps=10,
    verbose=False,
)

covariance matrix :
 [[ 1.          0.93545218  0.91636829 -0.91293008 -0.897829    0.89101795
  -0.79674341 -0.92193694]
 [ 0.93545218  1.          0.94658663 -0.94303505 -0.92743599  0.92040033
  -0.82301699 -0.95233891]
 [ 0.91636829  0.94658663  1.         -0.92379646 -0.90851563  0.90162351
  -0.80622685 -0.93291052]
 [-0.91293008 -0.94303505 -0.92379646  1.          0.90510689 -0.89824062
   0.80320189  0.92941025]
 [-0.897829   -0.92743599 -0.90851563  0.90510689  1.         -0.88338253
   0.78991586  0.91403656]
 [ 0.89101795  0.92040033  0.90162351 -0.89824062 -0.88338253  1.
  -0.78392346 -0.90710256]
 [-0.79674341 -0.82301699 -0.80622685  0.80320189  0.78991586 -0.78392346
   1.          0.81112619]
 [-0.92193694 -0.95233891 -0.93291052  0.92941025  0.91403656 -0.90710256
   0.81112619  1.        ]]
decision matrix :
 [[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]
expect

# 8 qubit graph state

In [13]:
# This expected matrix is not known with certainty, but agrees with results
graph8_mat = qnp.array(
    [
        [1, 0, 0, 1, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1],
    ]
)

graph8_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [0, 3], [6, 7]]),
)

In [14]:
characteristic_matrix_inference(graph8_state_prep_node, graph8_mat, num_steps=60, step_size=0.02)

iteration :  0 , score :  0.027825668671072057
elapsed time :  0.06241893768310547
iteration :  1 , score :  0.028609226815533795
elapsed time :  0.0607602596282959
iteration :  2 , score :  0.029421008868100396
elapsed time :  0.08008313179016113
iteration :  3 , score :  0.030262321166808492
elapsed time :  0.06405401229858398
iteration :  4 , score :  0.031134544835440536
elapsed time :  0.12206077575683594
iteration :  5 , score :  0.032039140796588095
elapsed time :  0.0651390552520752
iteration :  6 , score :  0.03297765515935147
elapsed time :  0.06704187393188477
iteration :  7 , score :  0.03395172501373045
elapsed time :  0.06212496757507324
iteration :  8 , score :  0.03496308466313747
elapsed time :  0.057868003845214844
iteration :  9 , score :  0.03601357233148139
elapsed time :  0.06325101852416992
iteration :  10 , score :  0.037105137382507314
elapsed time :  0.1110537052154541
iteration :  11 , score :  0.03823984809402292
elapsed time :  0.06394791603088379
iteration

In [20]:
covariance_matrix_inference(
    graph8_state_prep_node,
    8,
    graph8_mat,
    step_size=0.05,
    num_steps=30,
    verbose=False,
)

covariance matrix :
 [[ 1.00000000e+00 -8.32667268e-17 -2.46519033e-32 -9.63371307e-01
  -2.67599372e-17  8.33048101e-17 -2.77555756e-17 -5.55111512e-17]
 [-8.32667268e-17  1.00000000e+00 -9.99997414e-01  2.77555756e-17
  -1.40271336e-16 -5.55682762e-17  1.84889275e-32 -8.32667268e-17]
 [-2.46519033e-32 -9.99997414e-01  1.00000000e+00 -5.55111512e-17
   2.97468524e-17  5.55873178e-17 -2.46519033e-32  2.77555756e-17]
 [-9.63371307e-01  2.77555756e-17 -5.55111512e-17  1.00000000e+00
  -2.92490332e-17 -5.55682762e-17  5.55111512e-17 -2.77555756e-17]
 [-2.67599372e-17 -1.40271336e-16  2.97468524e-17 -2.92490332e-17
   9.99919577e-01  1.48176556e-16 -2.67599372e-17 -1.49345759e-18]
 [ 8.33048101e-17 -5.55682762e-17  5.55873178e-17 -5.55682762e-17
   1.48176556e-16  9.99999882e-01  5.55492345e-17 -5.55682762e-17]
 [-2.77555756e-17  1.84889275e-32 -2.46519033e-32  5.55111512e-17
  -2.67599372e-17  5.55492345e-17  1.00000000e+00 -9.99999405e-01]
 [-5.55111512e-17 -8.32667268e-17  2.77555756e-1

## 8-Qubit Graph State with Frustrated CNOT arrangement

In [16]:
graph8_cyclic_mat = qnp.array(
    [
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1],
    ]
)

graph8_cyclic_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [2, 0], [6, 7]]),
)

In [17]:
characteristic_matrix_inference(
    graph8_cyclic_state_prep_node, graph8_cyclic_mat, num_steps=60, step_size=0.02
)

iteration :  0 , score :  0.9963315424187213
elapsed time :  0.060235023498535156
iteration :  1 , score :  1.0346753749722342
elapsed time :  0.12036013603210449
iteration :  2 , score :  1.073425468786294
elapsed time :  0.06766629219055176
iteration :  3 , score :  1.1124937964991068
elapsed time :  0.06234288215637207
iteration :  4 , score :  1.1517891727958376
elapsed time :  0.06479263305664062
iteration :  5 , score :  1.1912178964432467
elapsed time :  0.07491183280944824
iteration :  6 , score :  1.2306844094104794
elapsed time :  0.0647730827331543
iteration :  7 , score :  1.2700919698319777
elapsed time :  0.12676572799682617
iteration :  8 , score :  1.3093433378082018
elapsed time :  0.06683015823364258
iteration :  9 , score :  1.3483414748805589
elapsed time :  0.07433176040649414
iteration :  10 , score :  1.386990259192364
elapsed time :  0.0664677619934082
iteration :  11 , score :  1.4251952186841967
elapsed time :  0.06216716766357422
iteration :  12 , score :  1.

In [15]:
covariance_matrix_inference(
    graph8_cyclic_state_prep_node,
    8,
    graph8_cyclic_mat,
    step_size=0.05,
    num_steps=30,
    verbose=False,
)

NameError: name 'graph8_cyclic_state_prep_node' is not defined

## 12-Qubit Graph State

In [24]:
graph12_mat = qnp.array(
    [
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
    ]
)
graph12_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [6, 7], [4, 5], [4, 9], [4, 11], [11, 10]]),
)

In [25]:
characteristic_matrix_inference(graph12_state_prep_node, graph12_mat, num_steps=60, step_size=0.02)

characteristic matrix :
 [[ 1.00000000e+00  1.03688023e-03  5.19980043e-01  3.77475828e-15
   3.99680289e-15  4.44089210e-15  4.44089210e-15  4.44089210e-15
   3.55271368e-15  2.88657986e-15  1.33226763e-15  2.22044605e-16]
 [ 1.03688023e-03  1.00000000e+00  3.72356509e-01  4.44089210e-15
   4.21884749e-15  3.10862447e-15  3.33066907e-15  4.44089210e-15
   4.44089210e-15  7.54951657e-15  6.21724894e-15 -3.77475828e-15]
 [ 5.19980043e-01  3.72356509e-01  1.00000000e+00  3.77475828e-15
   2.88657986e-15  5.77315973e-15  2.44249065e-15  2.88657986e-15
   4.21884749e-15  1.33226763e-15  1.33226763e-15  2.22044605e-16]
 [ 3.77475828e-15  4.44089210e-15  3.77475828e-15  5.58374350e-03
   4.21884749e-15  3.55271368e-15  4.21884749e-15  4.21884749e-15
   2.22044605e-15  8.43769499e-15  6.66133815e-16  1.59872116e-14]
 [ 3.99680289e-15  4.21884749e-15  2.88657986e-15  4.21884749e-15
   1.00000000e+00  2.75514555e-03  4.88498131e-15  4.44089210e-15
   7.10542736e-15  1.04659354e-02  5.77315973e-

In [18]:
covariance_matrix_inference(
    graph12_state_prep_node,
    12,
    graph12_mat,
    num_steps=20,
    step_size=0.05,
    verbose=False,
)

NameError: name 'graph12_state_prep_node' is not defined

## A bunch of different entangled states

In [17]:
multi_ent_mat_full = qnp.zeros((8, 8))
multi_ent_mat = qnp.array(
    [
        [1, 1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1],
    ]
)


def fn(settings, wires):
    qnetvo.shared_coin_flip_state([qnp.pi / 3], wires=[wires[0], wires[1], wires[7]])
    qnetvo.ghz_state([], wires=wires[2:4])
    qnetvo.W_state([], wires=wires[4:7])


multi_ent_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4, 5, 6, 7],
    ansatz_fn=fn,
)

In [18]:
characteristic_matrix_inference(
    multi_ent_prep_node, multi_ent_mat_full, num_steps=50, step_size=0.1
)

characteristic matrix :
 [[8.74939524e-01 5.88127854e-03 1.77635684e-15 1.11022302e-15
  2.88657986e-15 1.55431223e-15 1.11022302e-15 1.92255121e-01]
 [5.88127854e-03 8.71216879e-01 1.11022302e-15 1.55431223e-15
  1.11022302e-15 2.66453526e-15 1.55431223e-15 7.92576390e-01]
 [1.77635684e-15 1.11022302e-15 1.00000000e+00 9.99983971e-01
  1.77635684e-15 1.77635684e-15 1.77635684e-15 1.77635684e-15]
 [1.11022302e-15 1.55431223e-15 9.99983971e-01 1.00000000e+00
  1.33226763e-15 1.77635684e-15 4.44089210e-16 1.77635684e-15]
 [2.88657986e-15 1.11022302e-15 1.77635684e-15 1.33226763e-15
  9.45063523e-01 1.61880641e-01 3.49976187e-01 1.55431223e-15]
 [1.55431223e-15 2.66453526e-15 1.77635684e-15 1.77635684e-15
  1.61880641e-01 9.24317922e-01 3.48645227e-01 8.88178420e-16]
 [1.11022302e-15 1.55431223e-15 1.77635684e-15 4.44089210e-16
  3.49976187e-01 3.48645227e-01 9.95020269e-01 1.77635684e-15]
 [1.92255121e-01 7.92576390e-01 1.77635684e-15 1.77635684e-15
  1.55431223e-15 8.88178420e-16 1.7763

In [19]:
covariance_matrix_inference(
    multi_ent_prep_node,
    7,
    multi_ent_mat,
    meas_wires=[0, 1, 2, 3, 4, 5, 6],
    num_steps=20,
    step_size=0.05,
    verbose=False,
)

covariance matrix :
 [[ 7.65452409e-01  7.15336288e-01 -5.46396893e-17 -8.23952649e-17
  -1.09287579e-16 -4.33680869e-17 -9.75781955e-18]
 [ 7.15336288e-01  7.57591956e-01 -5.50864639e-17  4.24687291e-19
  -2.60208521e-17 -5.55111512e-17 -5.09575021e-17]
 [-5.46396893e-17 -5.50864639e-17  1.00000000e+00  9.99806369e-01
   2.68037026e-17  8.16312905e-17 -2.77272844e-17]
 [-8.23952649e-17  4.24687291e-19  9.99806369e-01  1.00000000e+00
   2.68037026e-17  8.16312905e-17 -2.77272844e-17]
 [-1.09287579e-16 -2.60208521e-17  2.68037026e-17  2.68037026e-17
   9.99705966e-01  6.61153830e-01  6.65736510e-01]
 [-4.33680869e-17 -5.55111512e-17  8.16312905e-17  8.16312905e-17
   6.61153830e-01  9.99132026e-01  6.63996315e-01]
 [-9.75781955e-18 -5.09575021e-17 -2.77272844e-17 -2.77272844e-17
   6.65736510e-01  6.63996315e-01  9.99999740e-01]]
decision matrix :
 [[1 1 0 0 0 0 0]
 [1 1 0 0 0 0 0]
 [0 0 1 1 0 0 0]
 [0 0 1 1 0 0 0]
 [0 0 0 0 1 1 1]
 [0 0 0 0 1 1 1]
 [0 0 0 0 1 1 1]]
expected matrix :
 [

## Finite Shot Examples

In [10]:
graph5_mat = qnp.array(
    [
        [1, 1, 1, 0, 0],
        [1, 1, 1, 0, 0],
        [1, 1, 1, 0, 0],
        [0, 0, 0, 1, 1],
        [0, 0, 0, 1, 1],
    ]
)

qnode_kwargs = {
    "diff_method": "parameter-shift",
}

graph5_state_prep_node = qnetvo.PrepareNode(
    wires=[0, 1, 2, 3, 4],
    ansatz_fn=qnetvo.graph_state_fn([[0, 1], [1, 2], [2, 0], [3, 4]]),
)

## 1000 Shots

In [11]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    num_steps=60,
    step_size=0.02,
    shots=1000,
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.99560686 0.07309891 0.01007241 0.00608938 0.00392606]
 [0.07309891 0.99440665 0.0045082  0.00535526 0.00461771]
 [0.01007241 0.0045082  0.99465832 0.00438815 0.00393425]
 [0.00608938 0.00535526 0.00438815 0.9962573  1.        ]
 [0.00392606 0.00461771 0.00393425 1.         0.99125401]]
decision matrix :
 [[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  2.3792134077268843


In [5]:
covariance_matrix_inference(
    graph5_state_prep_node,
    5,
    graph5_mat,
    step_size=0.2,
    num_steps=6,
    shots=1000,
    qnode_kwargs=qnode_kwargs,
)

covariance matrix :
 [[ 9.98704e-01  9.88632e-01  9.94848e-01  1.44000e-03 -1.44000e-03]
 [ 9.88632e-01  9.98556e-01  9.88784e-01 -4.80000e-04  4.80000e-04]
 [ 9.94848e-01  9.88784e-01  9.98976e-01  1.28000e-03 -1.28000e-03]
 [ 1.44000e-03 -4.80000e-04  1.28000e-03  9.98400e-01 -9.98400e-01]
 [-1.44000e-03  4.80000e-04 -1.28000e-03 -9.98400e-01  9.98400e-01]]
decision matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  0.024371485305577978


## 500 Shots

In [6]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    num_steps=40,
    step_size=0.1,
    shots=500,
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.99334181 0.00831194 0.99989612 0.00664245 0.00833094]
 [0.00831194 0.98958752 0.01929609 0.00911956 0.01120093]
 [0.99989612 0.01929609 0.99334181 0.00685853 0.00624924]
 [0.00664245 0.00911956 0.00685853 0.99156977 1.        ]
 [0.00833094 0.01120093 0.00624924 1.         0.99218378]]
decision matrix :
 [[1 0 1 0 0]
 [0 1 0 0 0]
 [1 0 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  1.9727129747696193


In [25]:
covariance_matrix_inference(
    graph5_state_prep_node,
    5,
    graph5_mat,
    step_size=0.2,
    num_steps=10,
    shots=500,
    qnode_kwargs=qnode_kwargs,
)

covariance matrix :
 [[ 0.995376  0.97192   0.995376 -0.030176  0.030176]
 [ 0.97192   0.9964    0.97192  -0.02192   0.02192 ]
 [ 0.995376  0.97192   0.995376 -0.030176  0.030176]
 [-0.030176 -0.02192  -0.030176  0.998976 -0.998976]
 [ 0.030176  0.02192   0.030176 -0.998976  0.998976]]
decision matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  0.11163914553596331


## 100 Shots

In [12]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    num_steps=40,
    step_size=0.1,
    shots=100,
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.94268319 1.         0.0588545  0.03092696 0.0575416 ]
 [1.         0.95067209 0.06485752 0.04181393 0.05033759]
 [0.0588545  0.06485752 0.94268319 0.10155949 0.05057845]
 [0.03092696 0.04181393 0.10155949 0.9248187  1.        ]
 [0.0575416  0.05033759 0.05057845 1.         0.93406806]]
decision matrix :
 [[1 1 0 0 0]
 [1 1 0 0 0]
 [0 0 1 1 0]
 [0 0 1 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  1.8927070986363297


In [13]:
covariance_matrix_inference(
    graph5_state_prep_node,
    5,
    graph5_mat,
    step_size=0.1,
    num_steps=10,
    shots=100,
    qnode_kwargs=qnode_kwargs,
    atol=0.3,
)

covariance matrix :
 [[ 0.9744  0.3032  0.9744 -0.08    0.08  ]
 [ 0.3032  0.9996  0.3032 -0.02    0.02  ]
 [ 0.9744  0.3032  0.9744 -0.08    0.08  ]
 [-0.08   -0.02   -0.08    1.     -1.    ]
 [ 0.08    0.02    0.08   -1.      1.    ]]
decision matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  1.413344459075706


## 50 Shots

In [16]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    num_steps=40,
    step_size=0.1,
    shots=50,
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.9248187  1.         1.         0.10584223 0.06048606]
 [1.         0.90438146 0.76748116 0.04350992 0.0582205 ]
 [1.         0.76748116 0.90438146 0.09561854 0.04260303]
 [0.10584223 0.04350992 0.09561854 0.90438146 1.        ]
 [0.06048606 0.0582205  0.04260303 1.         0.95804202]]
decision matrix :
 [[1 1 1 1 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [1 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  0.45296252193266145


In [13]:
covariance_matrix_inference(
    graph5_state_prep_node,
    5,
    graph5_mat,
    step_size=0.1,
    num_steps=10,
    shots=50,
    qnode_kwargs=qnode_kwargs,
    atol=0.4,
)

covariance matrix :
 [[ 1.      1.      1.     -0.08    0.08  ]
 [ 1.      1.      1.     -0.08    0.08  ]
 [ 1.      1.      1.     -0.08    0.08  ]
 [-0.08   -0.08   -0.08    0.9856 -0.9856]
 [ 0.08    0.08    0.08   -0.9856  0.9856]]
decision matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  0.2786206022533149


## 10 Shots

In [14]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    num_steps=40,
    step_size=0.1,
    shots=10,
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[0.72192809 0.3958156  0.97095059 0.3958156  0.55677965]
 [0.3958156  0.46899559 0.97095059 0.55677965 0.3958156 ]
 [0.97095059 0.97095059 0.46899559 0.41997309 0.41997309]
 [0.3958156  0.55677965 0.41997309 0.46899559 1.        ]
 [0.55677965 0.3958156  0.41997309 1.         0.72192809]]
decision matrix :
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  2.0751390139564574


In [15]:
covariance_matrix_inference(
    graph5_state_prep_node,
    5,
    graph5_mat,
    step_size=0.1,
    num_steps=10,
    shots=10,
    qnode_kwargs=qnode_kwargs,
    atol=0.3,
)

covariance matrix :
 [[ 0.96  0.08 -0.24 -0.24  0.24]
 [ 0.08  0.64  0.08  0.08 -0.08]
 [-0.24  0.08  0.96  0.56 -0.56]
 [-0.24  0.08  0.56  0.96 -0.96]
 [ 0.24 -0.08 -0.56 -0.96  0.96]]
decision matrix :
 [[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 1 1]
 [0 0 1 1 1]
 [0 0 1 1 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  2.4880514464134382


## 5 Shots


In [16]:
characteristic_matrix_inference(
    graph5_state_prep_node,
    graph5_mat,
    num_steps=40,
    step_size=0.1,
    shots=5,
    qnode_kwargs=qnode_kwargs,
)

characteristic matrix :
 [[-0.          0.97095059  0.72192809  0.97095059  0.72192809]
 [ 0.97095059 -0.          0.97095059  0.97095059  0.97095059]
 [ 0.72192809  0.97095059 -0.          0.97095059  0.97095059]
 [ 0.97095059  0.97095059  0.97095059 -0.          0.97095059]
 [ 0.72192809  0.97095059  0.97095059  0.97095059 -0.        ]]
decision matrix :
 [[0 1 1 1 1]
 [1 0 1 1 1]
 [1 1 0 1 1]
 [1 1 1 0 1]
 [1 1 1 1 0]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  3.9534190384647188


In [17]:
covariance_matrix_inference(
    graph5_state_prep_node,
    5,
    graph5_mat,
    step_size=0.1,
    num_steps=10,
    shots=5,
    qnode_kwargs=qnode_kwargs,
    atol=0.5,
)

covariance matrix :
 [[ 0.96  0.16  0.96 -0.16 -0.48]
 [ 0.16  0.96  0.16 -0.96 -0.48]
 [ 0.96  0.16  0.96 -0.16 -0.48]
 [-0.16 -0.96 -0.16  0.96  0.48]
 [-0.48 -0.48 -0.48  0.48  0.64]]
decision matrix :
 [[1 0 1 0 0]
 [0 1 0 1 0]
 [1 0 1 0 0]
 [0 1 0 1 0]
 [0 0 0 0 1]]
expected matrix :
 [[1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 1 1]
 [0 0 0 1 1]]
distance to expected :  2.6135033958271414
